In [1]:
import glob
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras import optimizers

C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from chat_wordvec import * 

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
maxsent = 50
num_features = 300

### Bentuk menjadi encoded form

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

def list_to_onehot(lista):
    global integer_encoded, onehot_encoded, end_dense_layer
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(lista)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    end_dense_layer = len(onehot_encoded)
    return onehot_encoded

def lst_to_onehot_encoded(lista,listb):
    result = []
    for i,la in enumerate(lista):
        cont = listb.index(la)
        print(i,la,cont,'->',onehot_encoded[cont], integer_encoded[cont])
        result.append(onehot_encoded[cont])
    return result
        
#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(onehot_encoded)

#print(onehot_encoded[aa.index("saya masih muda")])

In [5]:
def permutate_list(lista,listb):
    "Permutate 2 lists to two outputs with subs"
    tmp_x = []
    tmp_y = []
    for la in lista:
        for lb in listb:
            tmp_x.append(la)
            tmp_y.append(lb)
    return tmp_x, tmp_y

def unwrap_list(lista):
    "Unwrap sub of a list, to make a flat list"
    t_x = []
    for tx in lista:
        for t in tx:
            t_x.append(t)
    return t_x

def make_uniq_list(lista):
    return list(set(lista))

## PROCESS ALL TEXT FILES

In [6]:
fls = glob.glob('./dialogs/*.yml')

trn_x = []
trn_y = []

tmp_x = []
tmp_y = []


for fl in fls:
    with open(fl,'r') as f:
        txt = f.readlines()


    for i,tx in enumerate(txt):
        if tx[0] == "A":
            tmp_x.append(tx[3:-1])
        elif tx[0] == "B":
            tmp_y.append(tx[3:-1])
        else:
            tmp_x, tmp_y = permutate_list(tmp_x,tmp_y)
            trn_x.append(tmp_x)
            trn_y.append(tmp_y)
            tmp_x = []
            tmp_y = []

    trn_x = unwrap_list(trn_x)
    trn_y = unwrap_list(trn_y)

### Membuat encoding respons robot

In [10]:
respons = make_uniq_list(trn_y)
one_hot = list_to_onehot(respons)
train_y = lst_to_onehot_encoded(trn_y,respons)
train_y = np.array(train_y)

import pickle
pickle.dump(respons, open("respons.p","wb"))
pickle.dump(integer_encoded, open("integer_encoded.p","wb"))
pickle.dump(onehot_encoded, open("onehot_encoded.p",'wb'))

0 nama saya cipi 2 -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [8]
1 nama saya cipi 2 -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [8]
2 nama saya cipi 2 -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [8]
3 nama saya cipi 2 -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [8]
4 nama saya cipi 2 -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [8]
5 nama saya cipi 2 -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [8]
6 saya lagi santai saja 5 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [15]
7 saya lagi santai saja 5 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [15]
8 saya lagi santai saja 5 -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] [15]
9 saya berumur enam bulan 13 -> [0. 0. 0

### rubah semua menjadi matrix

In [11]:
for i,o in enumerate(respons):
    print(i, o, '->', one_hot[i])

0 saya senang berkenalan dengan anda -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0.]
1 kabar baik-baik saja. -> [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
2 nama saya cipi -> [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
3 saya masih muda -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0.]
4 terima kasih, saya juga senang -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1.]
5 saya lagi santai saja -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
6 saya suka ngobrol -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0.]
7 bumi itu bulat -> [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
8 senang berkenalan -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0.]
9 saya juga tertarik -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [12]:
print(respons)
print(integer_encoded)
print(onehot_encoded)

['saya senang berkenalan dengan anda', 'kabar baik-baik saja.', 'nama saya cipi', 'saya masih muda', 'terima kasih, saya juga senang', 'saya lagi santai saja', 'saya suka ngobrol', 'bumi itu bulat', 'senang berkenalan', 'saya juga tertarik', 'saya melayani majikan saya', 'saya terdiri dari syaraf otak digital dan kode pithon', 'saya tinggal disini saja', 'saya berumur enam bulan', 'saya di panggil cipi, panggil saja', 'hehehe', 'saya disini-sini saja', 'cuaca menurut saya sangat menarik, hangat dan semilir', 'asik banget nih', 'kasihan, apa yang terjadi', 'saya bisa bahasa indonesia, inggris dan komputer', 'di mesin komputer', 'he', 'saya suka kamu', 'sampai jumpa lagi']
[[18]
 [ 6]
 [ 8]
 [16]
 [24]
 [15]
 [20]
 [ 1]
 [23]
 [14]
 [17]
 [21]
 [22]
 [10]
 [12]
 [ 5]
 [13]
 [ 2]
 [ 0]
 [ 7]
 [11]
 [ 3]
 [ 4]
 [19]
 [ 9]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0

## cekidot data

In [13]:
for i,t in enumerate(trn_x):
    print(i, t, '->' , trn_y[i], train_y[i])

0 siapa kamu -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
1 apa panggilan kamu -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
2 nama -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
3 namanya siapa -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
4 siapa nama kamu -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
5 nama saya -> nama saya cipi [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
6 Kamu sedang apa -> saya lagi santai saja [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
7 Kamu lagi ngapain -> saya lagi santai saja [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
8 lagi apa -> saya lagi santai saja [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.

In [14]:
train_x = []
for tx in trn_x:
    train_x.append(swv_ar(twl(tx,rempunct=True,flat=True), maxword=maxsent, vecsize=num_features,frontpad=True))
train_x = np.array(train_x)

Problem at phrase: hehehe


In [15]:
print(train_x.shape)
print(train_y.shape)

(44, 50, 300)
(44, 25)


In [16]:
model=Sequential()
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="sigmoid", units=300, dropout=0.2))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=False, activation="sigmoid", units=300,dropout=0.2))
model.add(Dense(300, activation="relu"))
model.add(Dense(end_dense_layer,activation="sigmoid"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#cosine_proximity
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 25)                7525      
Total params: 1,540,225
Trainable params: 1,540,225
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_x,train_y, epochs=300)
model.save("./model/chitchat-300-1.mdl")

Epoch 1/300
44/44 [==============================] - 0s 9ms/step - loss: -0.8979 - acc: 0.8182
Epoch 2/300
44/44 [==============================] - 0s 7ms/step - loss: -0.8942 - acc: 0.8182
Epoch 3/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8940 - acc: 0.8409
Epoch 4/300
44/44 [==============================] - 0s 7ms/step - loss: -0.8998 - acc: 0.8182
Epoch 5/300
44/44 [==============================] - 0s 7ms/step - loss: -0.8945 - acc: 0.7955
Epoch 6/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8958 - acc: 0.8182
Epoch 7/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8978 - acc: 0.8182
Epoch 8/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8987 - acc: 0.7727
Epoch 9/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8989 - acc: 0.8409
Epoch 10/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8975 - acc: 0.8182
Epoch 11/300
44/44 [=============================

44/44 [==============================] - 0s 8ms/step - loss: -0.8992 - acc: 0.8182
Epoch 87/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9000 - acc: 0.8636
Epoch 88/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9010 - acc: 0.8409
Epoch 89/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8938 - acc: 0.8409
Epoch 90/300
44/44 [==============================] - 0s 9ms/step - loss: -0.8982 - acc: 0.8636
Epoch 91/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9017 - acc: 0.8409
Epoch 92/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8987 - acc: 0.8409
Epoch 93/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8976 - acc: 0.8409
Epoch 94/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8972 - acc: 0.8409
Epoch 95/300
44/44 [==============================] - 0s 8ms/step - loss: -0.8967 - acc: 0.8409
Epoch 96/300
44/44 [==============================] -

44/44 [==============================] - 0s 8ms/step - loss: -0.9050 - acc: 0.8409
Epoch 171/300
44/44 [==============================] - 0s 9ms/step - loss: -0.9062 - acc: 0.8182
Epoch 172/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9060 - acc: 0.8636
Epoch 173/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9063 - acc: 0.8182
Epoch 174/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9063 - acc: 0.8636
Epoch 175/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9066 - acc: 0.8182
Epoch 176/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9064 - acc: 0.8409
Epoch 177/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9066 - acc: 0.8409
Epoch 178/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9067 - acc: 0.8409
Epoch 179/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9066 - acc: 0.8636
Epoch 180/300
44/44 [=======================

44/44 [==============================] - 0s 8ms/step - loss: -0.9065 - acc: 0.8409
Epoch 255/300
44/44 [==============================] - 0s 7ms/step - loss: -0.9058 - acc: 0.8409
Epoch 256/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9065 - acc: 0.8409
Epoch 257/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9068 - acc: 0.8636
Epoch 258/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9064 - acc: 0.8182
Epoch 259/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9064 - acc: 0.8182
Epoch 260/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9065 - acc: 0.8409
Epoch 261/300
44/44 [==============================] - 0s 7ms/step - loss: -0.9065 - acc: 0.8409
Epoch 262/300
44/44 [==============================] - 0s 8ms/step - loss: -0.9060 - acc: 0.8409
Epoch 263/300
44/44 [==============================] - 0s 9ms/step - loss: -0.9067 - acc: 0.8409
Epoch 264/300
44/44 [=======================

In [20]:
def pred(sentence):
    """memprediksi masuk intent apakah sebuah kalimat."""
    predictions=model.predict(np.array([swv_ar(twl(sentence,rempunct=True,flat=True),maxword=maxsent, vecsize=num_features)]))
    cls_pred = np.argmax(predictions,axis=1)
    #print(predictions)
    
    #ini sangat tricky -> dari prediksi balikkan ke integer_encodednya dulu baru 
    #cari ke indexnya dari response
    
    jawaban = respons[list(integer_encoded).index(cls_pred[0])]

    result={'jawaban': jawaban , 'score': predictions.max(), 'sentence': sentence, 'cls:': cls_pred}
    return print(result)

In [21]:
for i in trn_x:
    pred(i)

{'jawaban': 'nama saya cipi', 'score': 0.30584776, 'sentence': 'siapa kamu', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.44346336, 'sentence': 'apa panggilan kamu', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.11785179, 'sentence': 'nama', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.37889957, 'sentence': 'namanya siapa', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.1913321, 'sentence': 'siapa nama kamu', 'cls:': array([8], dtype=int64)}
{'jawaban': 'nama saya cipi', 'score': 0.30314234, 'sentence': 'nama saya', 'cls:': array([8], dtype=int64)}
{'jawaban': 'saya lagi santai saja', 'score': 0.55342174, 'sentence': 'Kamu sedang apa', 'cls:': array([15], dtype=int64)}
{'jawaban': 'saya lagi santai saja', 'score': 0.22387679, 'sentence': 'Kamu lagi ngapain', 'cls:': array([15], dtype=int64)}
{'jawaban': 'saya lagi santai saja', 'score': 0.794692, 'sentence': 'lagi apa', 'cls

In [40]:
%%time
pred('ada apa')

{'jawaban': 'saya suka kamu', 'score': 0.9469023, 'sentence': 'ada apa', 'cls:': array([19], dtype=int64)}
Wall time: 14 ms
